In [1]:
## mashup script for batch changes
import math, json
from collections import Counter
from datetime import datetime, timedelta
from geopandas.tools import sjoin
# from pyproj import proj
import pandas as pd
import csv
import numpy as np
from shapely.geometry import Point
from shapely.geometry.polygon import Polygon
import matplotlib.pyplot as plt
from shapely.geometry import Point
import geopandas as gpd
import fiona
from fulcrum import Fulcrum

In [2]:
## variables
apiToken = "689a52b9420360bef6af63ad889bcbf4cb34f26e1d1348b2d3a2ff3b71dfe6828d07395724415aa9"
urlBase = 'https://api.fulcrumapp.com/api/v2/'
fulcrum = Fulcrum(key=apiToken)
osTimestamp = datetime.today() - timedelta(days=4)
recordsPerPage = 5000
print(fulcrum)

In [3]:
## data CTI
appID = "0092aa7f-a2e8-4972-8f26-46eb98b23a12" # ID of Combined Tree Invnetory
formdataSource = fulcrum.forms.find(appID)
# get number of pages
## the following divides the total number of records in chunks and downloads and assembles them. Otherwise the fulcrum API allows only downloads of 20,000 records

## count records in app
recordCountSource = fulcrum.forms.find(appID)['form']['record_count']
pagesSource = math.ceil(recordCountSource / recordsPerPage)

## get data I
if pagesSource > 1:
    for p in range(1, pagesSource + 1):
        dataPage = fulcrum.records.search(url_params={'form_id': appID, 'page': p, 'per_page': recordsPerPage})[
            'records']
        if p > 1:
            dataSource.extend(dataPage)
        else:
            dataSource = dataPage
else:
    dataSource = fulcrum.records.search(url_params={'form_id': appID, 'page': 1, 'per_page': recordsPerPage})['records']

In [4]:
## What do we have? Attributes:
type(dataSource)
cticolumns = list(dataSource[0].keys())
cticolumns

['status',
 'version',
 'id',
 'created_at',
 'updated_at',
 'client_created_at',
 'client_updated_at',
 'created_by',
 'created_by_id',
 'updated_by',
 'updated_by_id',
 'created_location',
 'updated_location',
 'created_duration',
 'updated_duration',
 'edited_duration',
 'form_id',
 'project_id',
 'record_series_id',
 'assigned_to',
 'assigned_to_id',
 'form_values',
 'latitude',
 'longitude',
 'altitude',
 'geometry',
 'speed',
 'course',
 'horizontal_accuracy',
 'vertical_accuracy',
 'system_status']

In [5]:
print(dataSource[0]["id"])
print(dataSource[1]["id"])
print(dataSource[2]["id"])
status_values = {record["id"] for record in dataSource if "id" in record}
len(status_values)

b65e1249-2b9f-42e5-bf6e-1e7449d5cda6
d6e275eb-04df-467e-805c-cdc965062b56
de09bd34-1dc6-4163-b1f0-aa3a4e3682ed


302745

In [6]:
import os
file_path = r"D:\TMO\Fulcrum\10032025_CTI\CTI_DQ_updated.csv"
print(os.path.exists(file_path))

True


In [26]:
import pandas as pd
CTIcsv = pd.read_csv(file_path, sep=";")
#print(CTIcsv)
print(CTIcsv["Project"].dropna().astype(str).value_counts())
print((CTIcsv["Project"].astype(str) == "DQ").sum())

Project
WS             42549
Qiddiya        38016
WH-Drone       29800
Al Hair        27370
WH-RCRC        23343
               ...  
RM-5               5
WOW                4
TR                 3
New Murabba        2
APR-T              1
Name: count, Length: 68, dtype: int64
22210


In [24]:
CTIcsv["id"]

0         66480c14-9661-46ff-9777-fdd19b2da9ec
1         601c7e15-11e9-4d11-8df7-a82035ba0b7f
2         046fdd5a-88bc-416a-85f8-97a08c7d325d
3         3f849168-ab42-4670-b771-c01c61554430
4         f844c4c7-6825-4808-ad20-2abe86f47993
                          ...                 
302712    251cd8a1-d386-4c66-ac80-428e54064956
302713    7f0c56b3-35d0-4176-859a-f5dd08aa0f2a
302714    de09bd34-1dc6-4163-b1f0-aa3a4e3682ed
302715    d6e275eb-04df-467e-805c-cdc965062b56
302716    b65e1249-2b9f-42e5-bf6e-1e7449d5cda6
Name: id, Length: 302717, dtype: object

In [14]:
print(dataSource[1]["project_id"])
print(dataSource[2]["project_id"])
print(dataSource[1]["id"])
print(dataSource[2]["id"])
print(dataSource[3]["id"])

8b0726a7-e56e-4230-874c-e55c63693caf
8b0726a7-e56e-4230-874c-e55c63693caf
d6e275eb-04df-467e-805c-cdc965062b56
de09bd34-1dc6-4163-b1f0-aa3a4e3682ed
7f0c56b3-35d0-4176-859a-f5dd08aa0f2a


In [33]:
## Lookup dictionary
cti_lookup = dict(zip(CTIcsv["id"], CTIcsv["Project"]))
print(list(cti_lookup.items())[:10])

[('66480c14-9661-46ff-9777-fdd19b2da9ec', 'NRR'), ('601c7e15-11e9-4d11-8df7-a82035ba0b7f', 'NRR'), ('046fdd5a-88bc-416a-85f8-97a08c7d325d', 'NRR'), ('3f849168-ab42-4670-b771-c01c61554430', 'NRR'), ('f844c4c7-6825-4808-ad20-2abe86f47993', 'NRR'), ('6f8b3286-1ff6-455d-b094-64e89165b06a', 'NRR'), ('7180992c-3a8f-4235-88df-692cd3ef339d', 'NRR'), ('720709a3-67a9-4960-9cf5-d6c18acad7ca', 'NRR'), ('36e77375-4ff2-4f60-96d1-f42c9db4e3b7', 'NRR'), ('1c1d8201-9342-46d2-9fc8-e291d7ba975a', 'NRR')]


In [37]:
filtered_cti_lookup = {k: v for k, v in cti_lookup.items() if v == "DQ"}
print(list(filtered_cti_lookup.items())[:10])

[('fe77f8c2-3263-499d-a228-4d0a9afae2f6', 'DQ'), ('61f82d6c-80e7-4bd3-9f43-09990235e9cb', 'DQ'), ('49e56190-9712-4835-ba4e-4179c3375578', 'DQ'), ('ac6d9164-7423-4637-ba5a-3285c3bc643e', 'DQ'), ('15bc2067-90a4-4212-a2c2-92f86221a510', 'DQ'), ('2be62438-5701-4833-b566-207a71587a31', 'DQ'), ('3f834781-454d-4685-9c9e-2da43b287c41', 'DQ'), ('b9153c2b-9bed-4465-b117-0ab0508143ea', 'DQ'), ('c58513cb-765f-46e6-8875-761dec7cd2b2', 'DQ'), ('15f95ddd-1121-4996-a699-9ffe972c4e67', 'DQ')]


In [45]:
## Update records by matching project_id with CTIcsv["id"]
for record in dataSource:
    record_id = record.get("id")  # Get record ID
    if record_id in filtered_cti_lookup and not record.get("Project"):
        try:
            # Fetch the full record from Fulcrum
            existing_record = fulcrum.records.find(record_id)

            # Update "Project" with "DQ" from filtered_cti_lookup
            existing_record["Project"] = "DQ"

            # Update the record in Fulcrum
            UPDATED = fulcrum.records.update(record_id, existing_record)

            print(f"Updated record {record_id} with Project: DQ")
        except Exception as e:
            print(f"Error updating record {record_id}: {e}")

Updated record 1c8a40de-d896-443b-9fba-91bade2d30c3 with Project: DQ
Updated record 333a6b3a-de89-487c-aee2-a51c6f40af6d with Project: DQ
Updated record 665c4938-5df8-4cf9-9743-cba0be10f072 with Project: DQ
Updated record 7ba8f26e-c5d0-43a4-8b15-0cc8921dea1d with Project: DQ
Updated record f7853bef-33d7-4ef6-81a9-f49d5e375a7f with Project: DQ
Updated record 3d8a3948-92e0-4cab-8ee6-fd6df52454db with Project: DQ
Updated record baf41855-c944-4fe3-b824-d807ce85547c with Project: DQ
Updated record 7422ef43-6fbc-4cbc-9dad-3c7b463f7049 with Project: DQ
Updated record d40facba-00a4-4b4f-a358-efddc3751146 with Project: DQ
Updated record 2a0dd0cc-8c5e-4eda-b68c-0a522f28ffbe with Project: DQ
Updated record 0de34c07-6a30-41b5-a297-1a157c30bec3 with Project: DQ
Updated record 5c6166a8-a5d1-4fe6-9e52-488bc46f3e1d with Project: DQ
Updated record 5971f356-6d1c-4028-8de0-e1784aa3a6f9 with Project: DQ
Updated record b461e746-f05e-4815-bcb7-a638ccf3179b with Project: DQ
Updated record 143e3d0b-41e4-47a9-

KeyboardInterrupt: 

In [44]:
UPDATED

{'record': {'status': 'No action required - لا حاجة لأي إجراء',
  'version': 6,
  'id': '333a6b3a-de89-487c-aee2-a51c6f40af6d',
  'created_at': '2023-11-09T11:29:04Z',
  'updated_at': '2025-03-11T09:38:07Z',
  'client_created_at': '2023-11-09T11:29:04Z',
  'client_updated_at': '2023-11-09T11:29:04Z',
  'created_by': '[BP] admin L2',
  'created_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1',
  'updated_by': '[BP] admin',
  'updated_by_id': 'd540ddf0-33db-4fa6-954c-edef0bb8ce30',
  'created_location': None,
  'updated_location': None,
  'created_duration': None,
  'updated_duration': None,
  'edited_duration': None,
  'form_id': '0092aa7f-a2e8-4972-8f26-46eb98b23a12',
  'project_id': None,
  'record_series_id': None,
  'changeset_id': None,
  'assigned_to': None,
  'assigned_to_id': None,
  'form_values': {'3fee': {'choice_values': ['Phoenix dactylifera - نخيل البلح'],
    'other_values': []},
   '6e5e': {'choice_values': ['ML-input'], 'other_values': []}},
  'latitude': 24.65998863,
  '

In [42]:
response = fulcrum.records.update(record_id, existing_record)
print(response)

{'record': {'status': 'No action required - لا حاجة لأي إجراء', 'version': 5, 'id': '5971f356-6d1c-4028-8de0-e1784aa3a6f9', 'created_at': '2023-11-09T11:30:37Z', 'updated_at': '2025-03-11T09:37:48Z', 'client_created_at': '2023-11-09T11:30:37Z', 'client_updated_at': '2023-11-09T11:30:37Z', 'created_by': '[BP] admin L2', 'created_by_id': 'f8161492-3026-4f3c-a770-93f8e540d4d1', 'updated_by': '[BP] admin', 'updated_by_id': 'd540ddf0-33db-4fa6-954c-edef0bb8ce30', 'created_location': None, 'updated_location': None, 'created_duration': None, 'updated_duration': None, 'edited_duration': None, 'form_id': '0092aa7f-a2e8-4972-8f26-46eb98b23a12', 'project_id': None, 'record_series_id': None, 'changeset_id': None, 'assigned_to': None, 'assigned_to_id': None, 'form_values': {'3fee': {'choice_values': ['Phoenix dactylifera - نخيل البلح'], 'other_values': []}, '6e5e': {'choice_values': ['ML-input'], 'other_values': []}}, 'latitude': 24.68922354, 'longitude': 46.61568399, 'altitude': None, 'geometry': 